In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold, train_test_split

# Step a: Load dataset
# Replace with your dataset path if needed
df = pd.read_csv('USA_Housing.csv')

# Separate input features (X) and target (y)
X = df.drop("Price", axis=1).values
y = df["Price"].values.reshape(-1, 1)

# Step b: Scale the input features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Step c: 5-fold cross-validation setup
kf = KFold(n_splits=5, shuffle=True, random_state=42)

best_beta = None
best_r2 = -np.inf  # initialize with very small value
r2_scores = []

# Step d: Perform 5-fold CV
for fold, (train_idx, test_idx) in enumerate(kf.split(X_scaled)):
    X_train, X_test = X_scaled[train_idx], X_scaled[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    # Add bias column of ones for intercept
    X_train_bias = np.c_[np.ones((X_train.shape[0], 1)), X_train]
    X_test_bias = np.c_[np.ones((X_test.shape[0], 1)), X_test]

    # Compute beta using Least Squares: β = (XᵀX)^(-1)Xᵀy
    beta = np.linalg.inv(X_train_bias.T @ X_train_bias) @ X_train_bias.T @ y_train

    # Predictions
    y_pred = X_test_bias @ beta

    # R2 score
    r2 = r2_score(y_test, y_pred)
    r2_scores.append(r2)
    print(f"Fold {fold+1} R2 Score:", r2)

    # Update best beta if this fold is better
    if r2 > best_r2:
        best_r2 = r2
        best_beta = beta

print("Best R2 from CV:", best_r2)

# Step e: Train-test split (70-30) using best beta
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

X_train_bias = np.c_[np.ones((X_train.shape[0], 1)), X_train]
X_test_bias = np.c_[np.ones((X_test.shape[0], 1)), X_test]

y_train_pred = X_train_bias @ best_beta
y_test_pred = X_test_bias @ best_beta

print("Final Train R2:", r2_score(y_train, y_train_pred))
print("Final Test R2:", r2_score(y_test, y_test_pred))

In [ ]:
# Train (56%), Validation (14%), Test (30%)
X_temp, X_test, y_temp, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.2, random_state=42)  # 0.2 of 70% = 14%

# Add bias term
X_train_b = np.c_[np.ones((X_train.shape[0], 1)), X_train]
X_val_b = np.c_[np.ones((X_val.shape[0], 1)), X_val]
X_test_b = np.c_[np.ones((X_test.shape[0], 1)), X_test]

# Gradient Descent Implementation
def gradient_descent(X, y, lr, n_iter=1000):
    m, n = X.shape
    beta = np.zeros((n, 1))
    for _ in range(n_iter):
        gradients = 2/m * X.T.dot(X.dot(beta) - y)
        beta -= lr * gradients
    return beta

learning_rates = [0.001, 0.01, 0.1, 1]
results = {}

for lr in learning_rates:
    beta = gradient_descent(X_train_b, y_train, lr)

    y_val_pred = X_val_b.dot(beta)
    y_test_pred = X_test_b.dot(beta)

    val_r2 = r2_score(y_val, y_val_pred)
    test_r2 = r2_score(y_test, y_test_pred)

    results[lr] = {"beta": beta, "val_r2": val_r2, "test_r2": test_r2}

# Display best result
best_lr = max(results, key=lambda x: results[x]["val_r2"])
print("Best learning rate:", best_lr)
print("Validation R2:", results[best_lr]["val_r2"])
print("Test R2:", results[best_lr]["test_r2"])
